# Towards new memory management in Text-Fabric

Text-Fabric uses a lot of RAM.

For a single user in a single process, this is no problem.

But if several processes on the same machine need to work with the data, it becomes extremely wasteful.

Example: a webserver with worker processes.

It would be nice to have the Text-Fabric data in **shared memory**, so that other python processes can use it.
Then we have a database in RAM, without the usual bottlenecks of interprocess communication.

[Shared memory](https://docs.python.org/3/library/multiprocessing.shared_memory.html#module-multiprocessing.shared_memory)
is in the multiprocessing library as from Python 3.8.

# Catch phrase

Build your own fast, in-memory text database with
[array](https://docs.python.org/3/library/array.html#module-array),
[bisect](https://docs.python.org/3/library/bisect.html#module-bisect),
[shared_memory](https://docs.python.org/3/library/multiprocessing.shared_memory.html#module-multiprocessing.shared_memory) and
[cython](https://cython.readthedocs.io/en/latest/index.html).

# The problem

Shared memory consists of raw bytes. One cannot place an arbitrary Python object there.

In particular, there is not yet a solution to place your dictionaries into shared memory.

There is a ShareableList class in the multiprocessing module, but it is still immature, and it is much slower than ordinary lists. See the [python bug tracker](https://bugs.python.org/issue38891).

But the generic shared memory really works, and if you are close to the metal, there is virtually no performance loss.
See [bare.ipynb](bare.ipynb).

Numpy works with shared memory, but Numpy is no good for storing strings and it does not deal with sparsity.
If use `numpy` arrays to do the same thing as I do with `array.array`, the performance suffers a bit.

# Why dicts?

In Text-Fabric features are mappings from integers to values, so you can also store feature data in lists.

However, many features are only defined for limited subranges of the nodes: the lists are sparsely filled.

We need a solution for sparse lists, and dictionaries form such a solution.

They are fast. However, they take up space, because the indices have to be stored as keys.

# Back to the metal

It is possible to implement sparse lists much closer to the metal, using the
[array](https://docs.python.org/3/library/array.html#module-array)
module in the standard library.

But then we need additional arithmetic, because feature values can have variable length.
We have to maintain offsets and lengths for all values.
Yet we achieve a 5-10 fold reduction in memory usage, and the otype feature drops from 8 MB to 180 **bytes**,
because there is really very little information in `otype`.

If we program this in pure python, we loose speed (think of a factor 7, for otype it is a factor 2-3).
We can use C-extension modules for part of that, such as 
[bisect](https://docs.python.org/3/library/bisect.html#module-bisect),
but there is still loss when we cross the boundaries between C and Python.

See [sparse.py](sparse.py) for the idea and [sparse.ipynb](sparse.ipynb) for the performance test.

# Cython

We can cythonize the logic.
My first attempts lead to an improvement of 20%. The performance loss w.r.t. to the dicts is then 4 fold.

See [cython.ipynb](cython.ipynb).

## Considerations

I just Cythonized Text-Fabric as a whole, without trying to add type declarations. 
That gave a mild performance boost: 1.5 as fast as before.

See [tfcython.ipynb](tfcython.ipynb).

I could try to get more out of it, especially in search, and maybe there are more aggressive speedups if both the feature access and the rest of Text-Fabric are Cythonized.

## Distribution

I do not know yet a good workflow to distribute a Cythonized Text-Fabric so that all users can `pip` install it.
But it can be done, lots of other modules out there are Cythonized.

# Opportunities

## One kernel, many users

Once we have a revamped memory management for text-fabric, new things come to mind:

A generic text-fabric kernel could act as a database for multiple datasets.
Via a console you can have it load or unload data.

Working with Text-Fabric then means: connect with a kernel. If the kernel has that data, you can work without any lag needed for the loading.

If you use the same dataset in multiple notebooks, they all use the same data.

## Binary data distribution

The binary data is very much shrunken. It might be a good idea to package that and attach it to a release of proper TF data. Users can then download that data and do not have to go through the process of precomputation anymore/

# Work

All this is a lot of work. Probably it will take two major versions, TF9 for the new shape of the data, and TF10 for utilizing it in a new type of TF kernel.

So far, I have only experimented with node features (both integer and string).
Edge features are more complicated, and also the precomputed data for the `L` (locality) API needs rethinking.

2020-07-02 CC-BY Dirk Roorda